## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,CityID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Feels like temp,Date
0,0,Buchanan,LR,5.8808,-10.0467,80.35,79,99,11.23,light rain,84.83,2022-06-15 10:56:17
1,1,East London,ZA,-33.0153,27.9116,61.74,44,0,31.07,clear sky,59.68,2022-06-15 10:50:51
2,2,Sao Filipe,CV,14.8961,-24.4956,74.35,82,94,8.21,overcast clouds,75.34,2022-06-15 10:49:34
3,3,Vaini,TO,-21.2000,-175.2000,71.76,94,75,4.61,broken clouds,73.06,2022-06-15 10:49:36
4,4,Albany,US,42.6001,-73.9662,61.30,87,17,2.53,few clouds,57.76,2022-06-15 10:46:39


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp=float(input('What\'s the max temperature you\'d like for your trip? '))
min_temp=float(input('What\'s the minimum temperature you\'d like for your trip? '))                     

What's the max temperature you'd like for your trip? 90
What's the minimum temperature you'd like for your trip? 70


In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_cities_df = city_data_df.loc[(city_data_df['Max Temp']<=max_temp)&(city_data_df['Max Temp']>=min_temp)]

filtered_cities_df.head(10)

,CityID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Feels like temp,Date
0,0,Buchanan,LR,5.8808,-10.0467,80.35,79,99,11.23,light rain,84.83,2022-06-15 10:56:17
2,2,Sao Filipe,CV,14.8961,-24.4956,74.35,82,94,8.21,overcast clouds,75.34,2022-06-15 10:49:34
3,3,Vaini,TO,-21.2000,-175.2000,71.76,94,75,4.61,broken clouds,73.06,2022-06-15 10:49:36
12,12,Salalah,OM,17.0151,54.0924,87.60,66,55,13.27,broken clouds,97.11,2022-06-15 10:50:53
13,13,Bitung,ID,1.4451,125.1824,79.65,82,78,7.07,broken clouds,79.65,2022-06-15 10:57:47
15,15,Esmeralda,CU,21.8525,-78.1169,74.64,85,1,5.68,clear sky,75.79,2022-06-15 10:57:47
16,16,Surt,LY,31.2089,16.5887,79.30,53,0,10.56,clear sky,79.30,2022-06-15 10:55:06
17,17,Port Hedland,AU,-20.3167,118.5667,73.45,68,25,8.05,scattered clouds,73.69,2022-06-15 10:57:48
19,19,Kavaratti,IN,10.5669,72.6420,83.35,75,100,22.08,light rain,90.61,2022-06-15 10:57:49
21,21,Goderich,CA,43.7501,-81.7165,71.92,53,23,10.36,few clouds,71.31,2022-06-15 10:57:49


In [13]:
# 4a. Determine if there are any empty rows.
filtered_cities_df.isna().sum()

CityID                 0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Feels like temp        0
Date                   0
dtype: int64

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = filtered_cities_df.dropna()

In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Buchanan,LR,80.35,light rain,5.8808,-10.0467,
2,Sao Filipe,CV,74.35,overcast clouds,14.8961,-24.4956,
3,Vaini,TO,71.76,broken clouds,-21.2000,-175.2000,
12,Salalah,OM,87.60,broken clouds,17.0151,54.0924,
13,Bitung,ID,79.65,broken clouds,1.4451,125.1824,
15,Esmeralda,CU,74.64,clear sky,21.8525,-78.1169,
16,Surt,LY,79.30,clear sky,31.2089,16.5887,
17,Port Hedland,AU,73.45,scattered clouds,-20.3167,118.5667,
19,Kavaratti,IN,83.35,light rain,10.5669,72.6420,
21,Goderich,CA,71.92,few clouds,43.7501,-81.7165,


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
#set params to search for a hotel and set base url since it doesn't change
# 6d. Set up the base URL for the Google Directions API to get JSON data.

base_url = r'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

#iterate through the df
# 6c. Get latitude and longitude from DataFrame
for index,row in hotel_df.iterrows():
    # get lat/long
    lat = row.Lat
    lng = row.Lng

    #add lat/lon to location key for params dict
    params['location'] = f'{lat},{lng}'    
    

    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.

    try:
        hotel_df.loc[index,'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print(f'Hotel not found in {row.City}....skipping.')

    

Hotel not found in Chokurdakh....skipping.
Hotel not found in Camacupa....skipping.
Hotel not found in Gambissara....skipping.
Hotel not found in Baykit....skipping.
Hotel not found in Poya....skipping.
Hotel not found in Saravan....skipping.
Hotel not found in Avera....skipping.
Hotel not found in Emba....skipping.
Hotel not found in Amapa....skipping.
Hotel not found in Effium....skipping.
Hotel not found in Katete....skipping.
Hotel not found in Tingi....skipping.
Hotel not found in Huayang....skipping.
Hotel not found in Jinji....skipping.
Hotel not found in Poum....skipping.
Hotel not found in Aykhal....skipping.
Hotel not found in Kizema....skipping.
Hotel not found in Thinadhoo....skipping.
Hotel not found in Krasnoarmeyskiy....skipping.
Hotel not found in Duvan....skipping.
Hotel not found in Cosereni....skipping.
Hotel not found in Gari....skipping.
Hotel not found in Verkhoyansk....skipping.
Hotel not found in Abu Samrah....skipping.
Hotel not found in Zachepylivka....skippin

In [28]:
# 7. Drop the rows where there is no Hotel Name.
#https://stackoverflow.com/questions/29314033/drop-rows-containing-empty-cells-from-a-pandas-dataframe
clean_hotel_df = hotel_df.replace('',pd.NA).dropna()

clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Buchanan,LR,80.35,light rain,5.8808,-10.0467,Kamaneahn Hotel Inc
2,Sao Filipe,CV,74.35,overcast clouds,14.8961,-24.4956,Hotel Xaguate
3,Vaini,TO,71.76,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
12,Salalah,OM,87.60,broken clouds,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
13,Bitung,ID,79.65,broken clouds,1.4451,125.1824,K2 Lembeh
...,...,...,...,...,...,...,...
667,Humberto De Campos,BR,77.59,few clouds,-2.5983,-43.4611,Pousada do Cabeça
671,Tukrah,LY,80.53,clear sky,32.5341,20.5791,Lagrota restaurant
674,Pahrump,US,73.45,clear sky,36.2083,-115.9839,Best Western Pahrump Oasis
675,Tarko-Sale,RU,71.80,overcast clouds,64.9225,77.7850,Yubileynaya


In [31]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [33]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [35]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))